# Quantum Reservoir Computing (QRC) for Swaption Pricing

This notebook implements a Hybrid Quantum-Classical machine learning model to predict Swaption prices. It utilizes a Quantum Reservoir Computing (QRC) approach, where a fixed quantum circuit (the reservoir) maps input features into a high-dimensional Hilbert space, followed by a trainable classical readout layer.

**Workflow:**
1. Data Loading & Preprocessing
2. Dimensionality Reduction (PCA)
3. Quantum Reservoir State Generation
4. Hybrid Model Training & Evaluation

## 1. Environment Setup & Dependencies
Installing necessary packages for quantum simulation (`merlinquantum`, `qiskit`, `pennylane`) and data handling.

In [ ]:
!pip install merlinquantum torch scikit-learn pandas numpy seaborn quantumreservoirpy

## 2. Imports & Configuration
Initializing global libraries and setting hyperparameters for the model (Modes, Photons, Epochs) and reproducible seeds.

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import os
import seaborn as sns
import pennylane as qml

from sklearn.metrics import r2_score, mean_squared_error


from quantumreservoirpy.reservoirs import Static, Incremental
from qiskit_aer import AerSimulator
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from merlin.core import PhotonicBackend as Experiment

from merlin import CircuitType
from merlin import StatePattern
# from merlin.core import AnsatzFactory

import perceval as pcvl
from merlin.algorithms import FeedForwardBlock
from merlin.measurement.strategies import MeasurementStrategy

from sklearn.decomposition import PCA


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import perceval as pcvl
import merlin
from merlin import QuantumLayer, ComputationSpace, LexGrouping, MeasurementStrategy
from merlin.builder import CircuitBuilder

In [ ]:
# ==========================================
# 2. Data Loading & Preprocessing
# ==========================================
torch.manual_seed(42)

TRAIN_FILE = "train.xlsx" # For future use

INCOMPLETE_PATH = 'price.xlsx'
# Model Hyperparameters
N_MODES =  8    # Number of optical modes (qubits equivalent)
N_PHOTONS = 4       # Number of photons
BATCH_SIZE = 32
EPOCHS = 50         # Adjust based on how fast your computer is
LEARNING_RATE = 0.005


## 3. Data Loading & Initial Inspection
Loading the raw training and pricing datasets to identify swaption tenors and maturities.

In [ ]:
#DATA LOADING and indentification
df_train = pd.read_excel(TRAIN_FILE)
df_train["Date"] = pd.to_datetime(df_train["Date"], format="%d/%m/%Y")

# Identify swaption columns
swaption_cols = [c for c in df_train.columns if c.startswith("Tenor")]

print("Training dataset loaded.")
print("Columns:", len(swaption_cols), "swaptions found.")


#Sample data
df_sample = pd.read_excel(INCOMPLETE_PATH)
df_sample["Date"] = pd.to_datetime(df_sample["Date"], format="%d/%m/%Y")


def parse_tenor_maturity(col):
    m = re.search(r"Tenor\s*:\s*([0-9.]+);\s*Maturity\s*:\s*([0-9.]+)", col)
    return float(m.group(1)), float(m.group(2))

## 4. Classical Baseline Models
Defining classical architectures to serve as a performance benchmark against the Quantum model.
1. **Linear Baseline:** A simple linear regression on raw features.
2. **PCA Baseline:** A linear regression using PCA-reduced features.

In [ ]:
# Simple Baseline Linear Model
class LinearModelBaseline(nn.Module):
    def __init__(self, image_size, num_classes=10):
        super(LinearModelBaseline, self).__init__()
        self.image_size = image_size

        # Classical part
        self.classifier = nn.Linear(image_size, num_classes)

    def forward(self, x):
        # Data is already flattened
        output = self.classifier(x)
        return output

In [ ]:
# LinearModel with PCA
class LinearModelPCA(nn.Module):
    def __init__(self, n_swaps, pca_components):
        super(LinearModelPCA, self).__init__()
        self.n_swaps = n_swaps
        self.pca_components = pca_components

        # Classical part
        self.classifier = nn.Linear(
            n_swaps+pca_components, n_swaps
        )
    def forward(self, x, x_pca):
        # Data is already flattened, just concatenate
        combined_features = torch.cat((x, x_pca), dim=1)
        output = self.classifier(combined_features)
        return output

## 5. Data Processing Pipeline
Here we define the `SwaptionDataset` class and the `load_and_process_data` function.

**Steps:**
1. **Target Engineering:** We predict the *difference* in price ($y_{t+1} - y_t$).
2. **Scaling:** Inputs are scaled to $[0, \pi]$ (for quantum angle encoding) and Targets are standardized.
3. **Splitting:** Data is split into training and testing sets based on time.

In [ ]:
#Data preparation
TRAIN_FILE = "train.xlsx"
SAMPLE_FILE = "sample_Simulated_Swaption_Price.xlsx"

#DATASET
class SwaptionDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def load_and_process_data(filepath):
    print(f"Loading data from {filepath}...")
    df = pd.read_excel(filepath)

    # Parse dates
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df = df.sort_values('Date').reset_index(drop=True)

    # Extract features (all columns except Date)
    feature_cols = [c for c in df.columns if c != 'Date']
    data_values = df[feature_cols].values

    print(f"Data shape: {data_values.shape} (Days, Surface Points)")

# Create Input (X) and Target (y)

    # --- SPLIT FIRST, THEN SCALE ---
    # We define the split index manually or using sklearn first

    X_raw = data_values[:-1]
    y_raw = data_values[1:]
    split_idx = int(len(X_raw) * 0.75)

    # y_target is now the DIFFERENCE (Change in price)
    y_diff = y_raw - X_raw 

    # Scale X (Input) normally
    scaler_x = MinMaxScaler(feature_range=(0, 1)) # Use 0,1 for stability
    X_scaled = scaler_x.fit_transform(X_raw)

    # Scale y_diff (Target) - Use StandardScaler for differences as they are centered around 0
    scaler_y = StandardScaler()
    y_diff_scaled = scaler_y.fit_transform(y_diff)
    
    X_train_raw, X_test_raw = X_raw[:split_idx], X_raw[split_idx:]
    y_train_raw, y_test_raw = y_raw[:split_idx], y_raw[split_idx:]

    # Define Scaler for the raw input/output (The curve data)
    # We fit ONLY on training data
    scaler = MinMaxScaler(feature_range=(0, np.pi))
    
    X_train_scaled = scaler.fit_transform(X_train_raw)
    # Transform test using train statistics
    X_test_scaled = scaler.transform(X_test_raw)
    
    y_train_scaled = scaler.transform(y_train_raw)
    y_test_scaled = scaler.transform(y_test_raw)

    return X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, scaler

# Load data
if not os.path.exists(TRAIN_FILE):
    raise FileNotFoundError(f"Could not find {TRAIN_FILE}. Please ensure it is in the folder.")

X_train, X_test, y_train, y_test, scaler = load_and_process_data(TRAIN_FILE)


# Create DataLoaders
train_loader = DataLoader(SwaptionDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(SwaptionDataset(X_test, y_test), batch_size=BATCH_SIZE)

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")

## 6. Quantum Reservoir Definitions

### Option A: Photonic Reservoir (Merlin)
Definition of a photonic quantum layer using the `merlinquantum` library.

In [ ]:
# definition of QuantumReservoir class - quantum layer applying on pca, and linear classifier on input and pca
class QuantumReservoir(nn.Module):
    def __init__(self, n_swaps, pca_components, n_modes, n_photons):
        #input_dim?
        super(QuantumReservoir, self).__init__()
        self.n_swaps = n_swaps
        self.pca_components = pca_components
        self.n_modes = n_modes
        self.n_photons = n_photons

        # Quantum part (non-trainable reservoir)
        self.quantum_layer = self._create_quantum_reservoir(
            pca_components, n_modes, n_photons
        )

        q_dim = self.quantum_layer.output_size

        # Classical part
        self.classifier = nn.Linear(
            q_dim+n_swaps, n_swaps)

        print(f"\nQuantum Reservoir Created:")
        print(f"  Input size (PCA components): {pca_components}")
        print(f"  Quantum output size: {self.quantum_layer.output_size}")
        print(f"  Total features to classifier: {n_swaps + self.quantum_layer.output_size}")

    def _create_quantum_reservoir(self, input_size, n_modes, n_photons):
        """Create quantum layer with Series circuit in reservoir mode."""
        builder = CircuitBuilder(n_modes=n_modes)

        builder.add_superpositions(depth=3)  # Deeper
        builder.add_angle_encoding()
        builder.add_rotations(trainable=False)  # Random fixed phases
        builder.add_superpositions(depth=3)
        builder.add_angle_encoding()
        quantum_layer = QuantumLayer(input_size=input_size, n_photons=n_photons, builder=builder,
                                     measurement_strategy=MeasurementStrategy.PROBABILITIES)        
        # builder.add_superpositions(depth=2)
        # builder.add_angle_encoding()
        # for _ in range(3):  # 3x deeper adaptive layers
        #     builder.add_adaptive_layers(num_layers=2, trainable=False)  # Random fixed params
        # builder.add_interferometers(depth=3)  # Multi-layer interferometry
        quantum_layer = QuantumLayer(input_size=input_size, n_photons=n_photons, builder=builder,
                                     measurement_strategy=merlin.MeasurementStrategy.PROBABILITIES,
                                     computation_space=ComputationSpace.FOCK)


        

        return quantum_layer

    def forward(self, x, x_pca):
        # Process the PCA-reduced input through quantum layer
        quantum_output = self.quantum_layer(x_pca)

        # Concatenate original image features with quantum output
        combined_features = torch.cat((x, quantum_output), dim=1)

        # Final classification
        output = self.classifier(combined_features)
        return output


### Option B: Gate-Based Reservoir (Qiskit/QRPy)
Definition of a gate-based superconducting circuit reservoir using `qiskit`. This class implements:
* **Data Re-uploading:** Encoding features into rotation gates ($R_y$) at multiple layers.
* **Entanglement:** CNOT gates to spread information across qubits.
* **Echo Dynamics:** Fixed random unitary evolution.

In [ ]:
# 1. Define the Reservoir Class Structure
class DeepStaticReservoir(Static):
    def __init__(self, n_qubits, n_layers, ry_angs, backend):
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.ry_angs = ry_angs
        super().__init__(n_qubits=n_qubits, backend=backend)

    def before(self, circuit):
        circuit.h(range(self.n_qubits))

    def during(self, circuit, timestep, reservoir_number):
        if np.isscalar(timestep):
            timestep = [timestep]

        # RE-UPLOADING ARCHITECTURE
        # Instead of encoding once at the start, we interleave encoding and processing
        
        for l in range(self.n_layers):
            # 1. ENCODING STEP (Data Re-uploading)
            # We map the features to the qubits again in every layer
            for i, feature_val in enumerate(timestep):
                qubit_idx = i % self.n_qubits
                # Use a specific scaling factor (e.g., 2.0 to cover full Bloch sphere)
                # Input assumed to be [-1, 1] or [0, 1] from scaler
                # Remove the generic * np.pi if your input is already [0, pi]
                # If input is [0, 1], use feature_val * 2 * np.pi
                circuit.ry(feature_val, qubit_idx) 

            # 2. PROCESSING STEP (Weights + Entanglement)
            for q in range(self.n_qubits):
                circuit.rx(self.ry_angs[l, q], q) # RX gives orthogonal rotation to RY
            
            # Entanglement (CNOTs)
            for q in range(self.n_qubits - 1):
                circuit.cx(q, q + 1)
            if self.n_qubits > 1:
                circuit.cx(self.n_qubits - 1, 0)

        # --- Reservoir Dynamics (Your existing "Echo" layers) ---
        for l in range(self.n_layers):
            for q in range(self.n_qubits):
                circuit.ry(self.ry_angs[l, q], q)
            # Full linear entanglement
            for q in range(self.n_qubits - 1):
                circuit.cx(q, q + 1)
            # Circular entanglement
            if self.n_qubits > 1:
                circuit.cx(self.n_qubits - 1, 0)

    def after(self, circuit):
        # circuit.measure_all()
        circuit.save_statevector()

class QRPyWrapper:
    def __init__(self, n_qubits, n_layers, backend):
        self.n_qubits = n_qubits
        self.backend = backend
        np.random.seed(42)
        self.ry_angs = np.pi * np.random.rand(n_layers, n_qubits)
        
        self.reservoir = DeepStaticReservoir(
            n_qubits=n_qubits, n_layers=n_layers, 
            ry_angs=self.ry_angs, backend=backend
        )

    def get_states(self, X_pca_data):
        input_timeseries = [row for row in X_pca_data]
        print("Running Quantum Simulation (Exact Statevector)...")
        # Run without shots (or shots=1 for statevector backend, but QRPy might handle it)
        # Note: We rely on the backend being configured for statevectors
        raw_states = self.reservoir.run(timeseries=input_timeseries) 
        return self._process_states(raw_states)

    def _process_states(self, res_states):
        # res_states will now contain Statevector objects if using save_statevector
        dim = 2**self.n_qubits
        feats = []
        
        for state in res_states:
            # Check if it is a Qiskit Statevector object
            if hasattr(state, 'data'): 
                # Get probabilities: |amplitude|^2
                probs = np.abs(state.data)**2
            elif isinstance(state, np.ndarray):
                probs = np.abs(state)**2
            else:
                # Fallback for dictionary counts (legacy)
                continue 
                
            # Truncate or Pad to ensure size consistency
            if len(probs) > dim:
                probs = probs[:dim]
            elif len(probs) < dim:
                probs = np.pad(probs, (0, dim - len(probs)))
                
            feats.append(probs)
            
        return np.array(feats, dtype=np.float32)


## 7. State Processing Utilities
Helper functions to extract probability distributions (statevectors) from the quantum backend and normalize them for the classical readout layer.

In [ ]:
def process_states(res_states, shots=1024, max_dim=256):
    """Safe extract: handles Counter or ndarray; normalize probs"""
    feats = []
    for state in res_states:
        if isinstance(state, dict) or hasattr(state, 'values'):  # Counter/dict
            counts = np.array(list(state.values()))
        elif isinstance(state, np.ndarray):
            counts = state.astype(int)
        else:
            raise ValueError(f"Unexpected state type: {type(state)}")
        
        probs = counts / np.sum(counts) if np.sum(counts) > 0 else np.zeros_like(counts)
        feats.append(probs[:max_dim])
    return np.array(feats, dtype=np.float32)


## 8. Feature Extraction (PCA)
To fit high-dimensional market data into limited quantum modes/qubits, we apply Principal Component Analysis (PCA). The features are then rescaled to $[0, 2\pi]$ for angle encoding.

In [ ]:
n_components=N_MODES*4
M=N_MODES
N=N_PHOTONS


pca = PCA(n_components=n_components)

# Note: Data is already flattened
X_train_flat = X_train
X_test_flat = X_test

X_train_pca_raw = pca.fit_transform(X_train_flat) # Assuming X_train_flat from Fix A
X_test_pca_raw = pca.transform(X_test_flat)

scaler_pca = MinMaxScaler(feature_range=(0, 2*np.pi))
X_train_pca = torch.FloatTensor(scaler_pca.fit_transform(X_train_pca_raw))
X_test_pca = torch.FloatTensor(scaler_pca.transform(X_test_pca_raw))

print(f"PCA Features Scaled Min: {X_train_pca.min()}, Max: {X_train_pca.max()}")
# Expected output: Min: 0.0, Max: 1.0


## 9. Reservoir State Generation
We pre-compute the quantum reservoir states. Since the reservoir weights are fixed (non-trainable), we can run the quantum simulation once for the entire dataset and cache the results to speed up training.

* **Backend:** `AerSimulator` (Statevector method).
* **Layers:** 10 layers of re-uploading and entanglement.

In [ ]:
# --- 1. Initialize Wrapper ---
n_qubits = 8
n_layers = 10 # Reduced from 20 for speed during debugging
backend = AerSimulator(method='statevector')
qr_wrapper = QRPyWrapper(n_qubits, n_layers, backend)


# --- 2. Pre-compute Reservoir States ---
# Ensure inputs are numpy
X_train_np = X_train_pca.numpy()
X_test_np = X_test_pca.numpy()

print("Generating Training States...")
feats_train_np = qr_wrapper.get_states(X_train_np)

print("Generating Test States...")
feats_test_np = qr_wrapper.get_states(X_test_np)

# Convert to Tensor
feats_train = torch.FloatTensor(feats_train_np)
feats_test = torch.FloatTensor(feats_test_np)

print(f"Reservoir Output Shape: {feats_train.shape}")

## 10. Hybrid Model Initialization
We define the `HybridReadout` model, which concatenates the original classical features (or PCA features) with the high-dimensional quantum reservoir features.

In [ ]:
class HybridReadout(nn.Module):
    def __init__(self, input_dim, reservoir_dim, output_dim):
        super().__init__()
        # Input: Original Features (or PCA) + Quantum Features
        self.layer = nn.Linear(input_dim + reservoir_dim, output_dim)
        
    def forward(self, x, q_feats):
        # Concatenate Classical and Quantum features
        combined = torch.cat((x, q_feats), dim=1)
        return self.layer(combined)

# Initialize Model
# input_dim can be X_train_tensor.shape[1] (original) or X_train_pca.shape[1] (PCA)
# Based on your previous code, you combined PCA + Quantum
model_input_dim = X_train_pca.shape[1] 
reservoir_dim = feats_train.shape[1]
output_dim = 224 

# Correctly initialize Baseline models with 224 outputs
linear_model = LinearModelBaseline(image_size=X_train_flat.shape[1], num_classes=output_dim)
pca_model = LinearModelPCA(n_swaps=output_dim, pca_components=n_components)

# Check dimensions
print(f"Linear Model Output Dim: {linear_model.classifier.out_features}") 
qrc_model = HybridReadout(model_input_dim, reservoir_dim, output_dim)
optimizer = torch.optim.Adam(qrc_model.parameters(), lr=0.0005)
loss_fn = nn.MSELoss()

## 11. Training & Evaluation
The final training loop for the hybrid model.
* **Optimizer:** Adam
* **Loss Function:** MSE
* **Metric:** $R^2$ Score on the testing set (inverse transformed to original price scale).

In [ ]:
# Create new DataLoaders that include the pre-computed reservoir features
train_dataset = torch.utils.data.TensorDataset(X_train_pca, feats_train, torch.Tensor(y_train))
# IMPORTANT: Shuffle MUST be True here for training, but we already aligned feats with X
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training Loop
epochs = 500
for epoch in range(epochs):
    qrc_model.train()
    running_loss = 0.0
    
    for batch_pca, batch_qfeats, batch_y in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        preds = qrc_model(batch_pca, batch_qfeats)
        loss = loss_fn(preds, batch_y)
        
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1} Loss: {running_loss/len(train_loader):.4f}")

# --- Evaluation Function with Fix ---
def evaluate_model(model, X_pca, Q_feats, y_true, scaler):
    model.eval()
    with torch.no_grad():
        preds_scaled = model(X_pca, Q_feats).numpy()
        y_true_scaled = y_true # Assuming y_true is passed as scaled tensor/numpy
    
    # 1. Inverse Transform Preds
    # The scaler expects 224 features. 
    # If the model predicts 224, direct inverse.
    if preds_scaled.shape[1] == scaler.n_features_in_:
        preds_real = scaler.inverse_transform(preds_scaled)
        # y_true might be tensor, convert to numpy
        if isinstance(y_true_scaled, torch.Tensor):
            y_true_scaled = y_true_scaled.numpy()
        y_real = scaler.inverse_transform(y_true_scaled)
    else:
        # Fallback for dimension mismatch (padding)
        # This shouldn't happen if output_dim=224, but good for safety
        padded_preds = np.zeros((preds_scaled.shape[0], scaler.n_features_in_))
        padded_preds[:, :preds_scaled.shape[1]] = preds_scaled
        preds_real = scaler.inverse_transform(padded_preds)[:, :preds_scaled.shape[1]]
        
        padded_y = np.zeros((y_true_scaled.shape[0], scaler.n_features_in_))
        padded_y[:, :y_true_scaled.shape[1]] = y_true_scaled
        y_real = scaler.inverse_transform(padded_y)[:, :y_true_scaled.shape[1]]

    # Metrics
    mse = mean_squared_error(y_real, preds_real)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_real, preds_real)
    
    print(f"Test MSE: {mse:.8f} | RMSE: {rmse:.4f} | R2: {r2:.4f}")
    return preds_real, y_real

# Run Eval
print("\nEvaluating...")
preds_real, y_real = evaluate_model(qrc_model, X_test_pca, feats_test, y_test, scaler)